# Import

In [16]:
import torch
import torch.nn as nn
from multiprocessing import Pool

import math, random, sys
import cPickle as pickle
import argparse

from fast_jtnn import *
import rdkit

# Write functions

In [17]:
def tensorize(smiles, assm=False):
    try: #added
        mol_tree = MolTree(smiles)
        mol_tree.recover()
        if assm:
            mol_tree.assemble()
            for node in mol_tree.nodes:
                if node.label not in node.cands:
                    node.cands.append(node.label)

        del mol_tree.mol
        for node in mol_tree.nodes:
            del node.mol
            del node.clique

        return mol_tree
    except: #added
        print smiles
        return None

In [38]:
def tensorize_pair(smiles_pair):
    mol_tree0 = tensorize(smiles_pair[0], assm=False) #added
    #mol_tree0 = tensorize(smiles_pair[0], assm=False)
    #mol_tree1 = tensorize(smiles_pair[1], assm=False) #added
    mol_tree1 = tensorize(smiles_pair[1], assm=False)
    return (mol_tree0, mol_tree1)

# Manually set args

In [19]:
ncpu = 12

In [26]:
train = '/home/rishi/g2g/improved/data/train.txt'

# Manually run main for pair

In [27]:
lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

#parser = argparse.ArgumentParser()
#parser.add_argument('--train', required=True)
#parser.add_argument('--mode', type=str, default='pair')
#parser.add_argument('--ncpu', type=int, default=8)
#args = parser.parse_args()

#ncpu = args.ncpu

In [28]:
pool = Pool(ncpu)

In [29]:
#train = args.train
#if args.mode == 'pair':
    #dataset contains molecule pairs
with open(train) as f:
    data = [line.strip("\r\n ").split()[:2] for line in f]

In [39]:
len(data)

3051

In [40]:
data[0:5]

[['CCCCOc1ccc(C=C2CCC(=Cc3ccc(OCCCCOc4ccc(C=C5CCC(=Cc6ccc(OCCCCOc7ccc(C=C8CCC(=Cc9ccc(O)cc9)C8=O)cc7)cc6)C5=O)cc4)cc3)C2=O)cc1',
  'CCCCCCCCOCCCCCCCCOCCCCCCCCO'],
 ['CCCCCc1ccc(CCCCCOC(=O)c2ccc(C(=O)OCCCCCc3ccc(CCCCCOC(=O)c4ccc(C(=O)OCCCCCc5ccc(CCCCCOC(=O)c6ccc(C(=O)O)cc6)cc5)cc4)cc3)cc2)cc1',
  'CCOC(=O)NC1CCC(NC(=O)OCCOC(=O)NC2CCC(NC(=O)OCCOC(=O)NC3CCC(NC(=O)O)CC3)CC2)CC1'],
 ['CCCCCCOc1csc(-c2sc(-c3scc(C)c3OCCCCCC)c(C)c2OCCCCCC)c1C',
  'CCCCCNC(=O)OCCCCCNC(=O)OCCCCCNC(=O)O'],
 ['CCCCOC(=O)SSSC(=O)OCCCCOC(=O)SSSC(=O)OCCCCOC(=O)SSSC(=O)O',
  'CCCCCCCCOC(=O)CCCCCCC(=O)OCCCCCCCCOC(=O)CCCCCCC(=O)OCCCCCCCCOC(=O)CCCCCCC(=O)O'],
 ['CCOCCOCCOCCOCCOc1ccc(C(=O)Nc2cc(NC(=O)c3ccc(OCCOCCOCCOCCOCCOc4ccc(C(=O)Nc5cc(NC(=O)c6ccc(OCCOCCOCCOCCOCCOc7ccc(C(=O)Nc8cc(NC(=O)c9ccc(O)cc9)cc(-c9nc%10ccccc%10[nH]9)c8)cc7)cc6)cc(-c6nc7ccccc7[nH]6)c5)cc4)cc3)cc(-c3nc4ccccc4[nH]3)c2)cc1',
  'CCOCCOC(=O)CCC(=O)OCCOCCOC(=O)CCC(=O)OCCOCCOC(=O)CCC(=O)O']]

In [41]:
#all_data = pool.map(tensorize_pair, data)
all_data1 = pool.map(tensorize_pair, data) #added

In [42]:
len(all_data1)

3051

In [43]:
all_data1

[(<fast_jtnn.mol_tree.MolTree at 0x7ff8a38da8d0>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a3b2f5d0>),
 (<fast_jtnn.mol_tree.MolTree at 0x7ff8a3a06050>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a3b8b750>),
 (<fast_jtnn.mol_tree.MolTree at 0x7ff8a3c57b50>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a3bef690>),
 (<fast_jtnn.mol_tree.MolTree at 0x7ff8a396a410>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a395ed50>),
 (<fast_jtnn.mol_tree.MolTree at 0x7ff8a38df8d0>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a38e6dd0>),
 (<fast_jtnn.mol_tree.MolTree at 0x7ff8a38f3510>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a38f5e50>),
 (<fast_jtnn.mol_tree.MolTree at 0x7ff8a38fc610>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a3880f50>),
 (<fast_jtnn.mol_tree.MolTree at 0x7ff8a38861d0>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a388b710>),
 (<fast_jtnn.mol_tree.MolTree at 0x7ff8a3899290>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a38a3410>),
 (<fast_jtnn.mol_tree.MolTree at 0x7ff8a38ac450>,
  <fast_jtnn.mol_tree.MolTree at 0x7ff8a3

In [35]:
all_data = [i for i in all_data1 if None not in i] #added

In [36]:
len(all_data)

3051

In [37]:
num_splits = len(all_data) / 100 #added
num_splits

30

In [20]:
if num_splits == 0: #added
    num_splits = 1 #added

In [21]:
le = (len(all_data) + num_splits - 1) / num_splits
le

182

In [22]:
for split_id in xrange(num_splits):
    st = split_id * le
    sub_data = all_data[st : st + le]

    with open('tensors-%d.pkl' % split_id, 'wb') as f:
        pickle.dump(sub_data, f, pickle.HIGHEST_PROTOCOL)

# elif args.mode == 'single':
#     #dataset contains single molecules
#     with open(args.train) as f:
#         data = [line.strip("\r\n ").split()[0] for line in f]

#     all_data = pool.map(tensorize, data)
#     num_splits = len(data) / 10000

#     le = (len(all_data) + num_splits - 1) / num_splits

#     for split_id in xrange(num_splits):
#         st = split_id * le
#         sub_data = all_data[st : st + le]

#         with open('tensors-%d.pkl' % split_id, 'wb') as f:
#             pickle.dump(sub_data, f, pickle.HIGHEST_PROTOCOL)

# Manually run main for single

Manually set args

In [22]:
train = '/home/rgur/CS6250_project/g2g/polymers_trial2/data/trial_test_899.txt'

In [23]:
#args.train = train #added

In [24]:
with open(train) as f:
    data = [line.strip("\r\n ").split()[0] for line in f]

In [25]:
#all_data = pool.map(tensorize, data)
all_data1 = pool.map(tensorize, data) #added

In [26]:
len(all_data1)

899

In [27]:
all_data = [i for i in all_data1 if i is not None] #added

In [28]:
len(all_data)

607

In [29]:
num_splits = len(all_data) / 10000 #added

In [30]:
if num_splits == 0: #added
    num_splits = 1 #added

In [31]:
le = (len(all_data) + num_splits - 1) / num_splits

In [32]:
le

607

In [33]:
for split_id in xrange(num_splits):
    st = split_id * le
    sub_data = all_data[st : st + le]

    with open('tensors-%d.pkl' % split_id, 'wb') as f:
        pickle.dump(sub_data, f, pickle.HIGHEST_PROTOCOL)